In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [102]:
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import re
import tensorflow as tf

I will use the functionality of Image Datasource of spark to collect and process images in bytes, and then start processing the images for the neural network

In [16]:
images_folders = ['/CA1/Ireland','/CA1/Honduran']

In [17]:
images_rdd = sc.binaryFiles(','.join(images_folders))

In [29]:
def extract_data(data):
    file_path, image_data = data
    image = Image.open(BytesIO(image_data))
    image_array = np.array(image)
    
    file_name = file_path.split('/')[-1]
    file_name_without_ext = file_name.split(".")[0]
    label,name = file_name_without_ext.split('_')[0], file_name_without_ext.split('_')[1]
    
    return name, label, image_array

In [30]:
imageDf = images_rdd.map(lambda x: extract_data(x)).toDF(["Name","Label","Data"])

NameError: name 'images_rdd' is not defined

In [29]:
imageDf.show()

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 193, in manager
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 874, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError


+----+-------+--------------------+
|Name|  Label|                Data|
+----+-------+--------------------+
| 001|Ireland|[FF D8 FF E0 00 1...|
| 002|Ireland|[FF D8 FF E0 00 1...|
| 003|Ireland|[FF D8 FF E0 00 1...|
| 004|Ireland|[FF D8 FF E0 00 1...|
| 005|Ireland|[FF D8 FF E0 00 1...|
| 006|Ireland|[FF D8 FF E0 00 1...|
| 007|Ireland|[FF D8 FF E0 00 1...|
| 008|Ireland|[FF D8 FF E0 00 1...|
| 009|Ireland|[FF D8 FF E0 00 1...|
| 010|Ireland|[FF D8 FF E0 00 1...|
| 011|Ireland|[FF D8 FF E0 00 1...|
| 012|Ireland|[FF D8 FF E0 00 1...|
| 013|Ireland|[FF D8 FF E0 00 1...|
| 014|Ireland|[FF D8 FF E0 00 1...|
| 015|Ireland|[FF D8 FF E0 00 1...|
| 016|Ireland|[FF D8 FF E0 00 1...|
| 017|Ireland|[FF D8 FF E0 00 1...|
| 018|Ireland|[FF D8 FF E0 00 1...|
| 019|Ireland|[FF D8 FF E0 00 1...|
| 020|Ireland|[FF D8 FF E0 00 1...|
+----+-------+--------------------+
only showing top 20 rows



In [35]:
pandasImagesDF = imageDf.toPandas()

Saving dataframe in a CSV, to not process the images each time I start the proyect

In [38]:
pandasImagesDF.to_csv("./Dataset/images.csv", index=False)

Read CSV saved in the project, to save time

In [2]:
pandasImagesDF = pd.read_csv("./Dataset/images.csv")

In [4]:
pandasImagesDF = pandasImagesDF.sort_values(by=["Name"])

In [36]:
pandasImagesDF

,Name,Label,Data
0,1,Ireland,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
20,1,Honduran,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
1,2,Ireland,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
21,2,Honduran,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
2,3,Ireland,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
22,3,Honduran,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
3,4,Ireland,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
23,4,Honduran,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
4,5,Ireland,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...
24,5,Honduran,bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...


In [37]:
def convert_string_to_np(string_repr):
    # Extract hexadecimal values from the string representation
    hex_values = re.findall(r'\\x([0-9a-fA-F]{2})', string_repr)
    # Convert hexadecimal values to bytes
    byte_data = bytes.fromhex(''.join(hex_values))
    # Convert bytes to NumPy array
    np_array = np.frombuffer(byte_data, dtype=np.uint8)
    return np_array

In [41]:
pandasImagesDF["Data"] = pandasImagesDF["Data"].apply(convert_string_to_np)

In [43]:
pandasImagesDF

,Name,Label,Data
0,1,Ireland,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
20,1,Honduran,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
1,2,Ireland,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
21,2,Honduran,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
2,3,Ireland,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
22,3,Honduran,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
3,4,Ireland,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
23,4,Honduran,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
4,5,Ireland,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."
24,5,Honduran,"[255, 216, 255, 224, 0, 16, 0, 1, 1, 0, 0, 1, ..."


Importing from Keras functionality necessary to implement CNN

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras import utils
from sklearn.metrics import accuracy_score

For Now there is only 20 photos per Category, the training size will be 28 and test 12, This is for testing. After this I'll get more images to improve the CNN

In [117]:
X_train = pandasImagesDF.iloc[:-12,2].values
y_train = pandasImagesDF.iloc[:-12,1].values
X_test = pandasImagesDF.iloc[-12:,2].values
y_test = pandasImagesDF.iloc[-12:,1].values


In [118]:
from sklearn.preprocessing import OneHotEncoder

In [119]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [120]:
X_train

array([array([0.00392157, 0.0033218 , 0.00392157, ..., 0.00244521, 0.00392157,
              0.00333718])                                                    ,
       array([0.00392157, 0.0033218 , 0.00392157, ..., 0.00030757, 0.00392157,
              0.00333718])                                                    ,
       array([0.00392157, 0.0033218 , 0.00392157, ..., 0.00258362, 0.00392157,
              0.00333718])                                                    ,
       array([0.00392157, 0.0033218 , 0.00392157, ..., 0.00030757, 0.00392157,
              0.00333718])                                                    ,
       array([0.00392157, 0.0033218 , 0.00392157, ..., 0.00253749, 0.00392157,
              0.00333718])                                                    ,
       array([0.00392157, 0.0033218 , 0.00392157, ..., 0.00269127, 0.00392157,
              0.00333718])                                                    ,
       array([3.92156863e-03, 3.32179931e-03, 

In [121]:
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)

In [122]:
X_train.shape

(28, 1)

In [123]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
y_train = np.reshape(y_train, (y_train.shape[0], y_train.shape[1],1))
y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1],1))

# X_train = X_train.astype("float32")
# X_test = X_test.astype("float32")

Because the pixels are from 0 to 255, We have to normalize the pixels

In [124]:
X_train/=255
X_test/=255

In [114]:
model = Sequential()
model.add(Conv2D(25,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu", input_shape=(1080,1080,1)))
model.add(MaxPool2D(pool_size=(1,1)))
model.add(Flatten())

model.add(Dense(100, activation="relu"))
model.add(Dense(10,activation="softmax"))

In [115]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [126]:
type(X_train)

numpy.ndarray

In [125]:
model.fit(X_train, y_train, batch_size=14, epochs=10, validation_data=(X_test, y_test))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).